# Biblioteka pandas w Pythonie

[Pandas](https://pandas.pydata.org/) to rozbudowana biblioteka do manipulowania danymi, tj. procesu pobierania danych i zmieniania ich formatu celem  łatwiejszego odczytu lub lepszego uporządkowania, oraz do analizy danych w Pythonie. 

Nazwa pochodzi od "**pan**el **da**ta", terminu powszechnie używanego w odniesieniu do wielowymiarowych zbiorów danych spotykanych w statystyce i ekonometrii.

**Uwaga**: Biblioteka pandas jest doskonałym narzędziem do pracy z małymi i średnimi zbiorami danych. Jednak zdolność pandas do przetwarzania dużych zbiorów danych jest ograniczona. Dzieje się tak, ponieważ pandas ładuje cały zbiór danych do pamięci RAM przed przetwarzaniem, co może być problematyczne, jeśli rozmiar zbioru danych przekracza dostępną pamięć.

## 0. Instalacja oraz import biblioteki pandas

Instalujemy za pomocą komendy pip:

In [ ]:
!pip install pandas

oraz importujemy za pomocą instrukcji

In [1]:
import pandas as pd

# Przyda się nam też NumPy, więc od razu i tę biblotekę zaimportujemy
import numpy as np

## 1. Podstawowe struktury danych w pandas

Serie danych (klasa [pandas.Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html)) – to jednowymiarowa tablica z etykietami, która może przechowywać dowolny typ danych.

Ramka danych (klasa [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)) – dwuwymiarowa struktura z etykietami, mogąca przechowywać kolumny z różnymi typami danych.

Indeks (klasa [pandas.Index](https://pandas.pydata.org/docs/reference/api/pandas.Index.html#pandas.Index)) - niezmienialny ciąg obiektów używany do indeksowania serii i ramek danych. 

### Seria i indeks w pandas

Serię danych możemy utworzyć w następujący sposób:

In [2]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

W powyższym przykładzie kolejne wartości są indeksowane automatycznie:

In [3]:
list(s.index)

[0, 1, 2, 3, 4, 5]

Oczywiście możemy określić swój własny sposób indeksowania wartości: 

In [4]:
s = pd.Series([12, 23, 19, 20], index=['Ala', 'Ola', 'Marek', 'Tomek'], dtype='int32')
s

Ala      12
Ola      23
Marek    19
Tomek    20
dtype: int32

Indeks pozwala nam odczytywać poszczególne elementy serii danych.

Na przykład tak:

In [5]:
s['Marek']

np.int32(19)

Ale można i tak, korzystając z odpowiedniego atrybutu:

In [6]:
s.Marek

np.int32(19)

Możemy też odczytywać dane z serii danych w "tradycyjny" sposób, tj. tak jak w przypadku standardowych list w Pythonie czy tabel w NumPy:

In [ ]:
s[0]

choć w przyszłości najprawdopodobniej zostanie to zmienione (w wersji 2.2.3 pojawia się odpowiednie ostrzeżenie).

In [ ]:
s[1:3]

W prosty sposób możemy stworzyć serię danych dla odczytów w określonych datach, punktach czasowych, itp. Tworzymy indeks dla poszczególnych odczytów, np. dla pierwszego dnia każdego miesiąca w 2020r.: 

In [ ]:
daty = pd.date_range('20200101', periods=12, freq='MS')
print(daty)

'MS' jest napisem określającym częstotliwość z jaką mają być generowane kolejne elementy zakresu. Możliwe inne wartości tego parametru są dostępne [tutaj](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases).

Utworzymy teraz serię danych zawierającą średnie wartości temperatury powietrza w Warszawie w poszczególnych miesiącach 2020r. zmierzone przez stację meteorologiczną Warszawa-Filtry (dla zainteresowanych: dane są dostępne [tutaj](https://meteomodel.pl/dane/srednie-miesieczne/?imgwid=252200230&par=tm&max_empty=2)):

In [ ]:
temp = pd.Series([2.7, 4.2, 5.4, 10.0, 12.8, 20.0, 20.3, 21.3, 16.0, 10.8, 6.0, 2.1], index=daty)
temp

Poszczególne wartości możemy odczytwać z serii danych na różne sposoby.

Pojedynczy element:

In [ ]:
temp[0]

In [ ]:
temp['2020-01-01']

In [ ]:
temp['2020/5/1']

In [ ]:
temp['2020-01']

Ale to już nie zadziała:

In [ ]:
temp.2020-01-01

ani to:

In [ ]:
temp.'2020-01-01'

Ale możemy tak:

In [ ]:
ts = pd.to_datetime('2020-01-01')
ts

i jeszcze na kilka różnych sposobów:

In [ ]:
ts = pd.to_datetime('2020-01')
ts

In [ ]:
pd.to_datetime('2020')

In [ ]:
temp[ts]

In [ ]:
temp['2020']

In [ ]:
temp['2020-05-01':'2020-08-01']

Co ciekawe, można i tak:

In [ ]:
temp['2020-05-01':'2020-08-12']

W naszym przykładzie serii danych zawierających średnie wartości temperatur dla poszczególnych miesięcy, etykiety indeksu są jednak mylące, gdyż zawierają informację o dniu - sugerują jakby pomiary były dokonywane w pierwszym dniu każdego miesiąca, co nie jest zgodne ze stanem faktycznym. Dlatego spróbujemy poprawić nasz indeks. Możemy zrobić to używająć funkcji [period_range](https://pandas.pydata.org/docs/reference/api/pandas.period_range.html). Najpierw utworzymy nowy indeks:

In [ ]:
nowy_indeks = pd.period_range("2020/01/01", freq="M", periods=12)
nowy_indeks

Zauważmy, że przy okazji zmienimy klasę indeksu z [DatetimeIndex](https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html) na [PeriodIndex](https://pandas.pydata.org/docs/reference/api/pandas.PeriodIndex.html).
Teraz dokonamy podmiany:

In [ ]:
temp.index = nowy_indeks
temp

Przy okazji nie tylko mamy bardziej poprawny indeks, ale też zredukowaliśmy zużycie pamięci:

In [ ]:
print(f"Niepoprawny indeks zajmował {daty.memory_usage()} bajtów.")
print(f"Poprawny indeks zajmuje {nowy_indeks.memory_usage()} bajtów.")

In [ ]:
daty.dtype

In [ ]:
nowy_indeks.dtype

Do zagadnień związanych z czasem i manipulowaniem nim w pandas jeszcze wrócimy na końcu dzisiejszych zajęć.

Omówimy jeszcze metodę [pandas.Series.map](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) dla serii danych. Pozwala ona na zastępowanie wartości serii danych innymi wartościami zadanymi przez funkcję, słownik lub inną serię danych:

In [ ]:
s = pd.Series(['cat', 'dog', np.nan, 'rabbit'])
s

Najpierw nowe wartości zadamy za pomocą słownika. Jeżeli w słowniku brakuje odpowiedniego klucza, to nową wartością jest NaN, chyba że słownik określa wartość domyślną:

In [ ]:
s.map({'cat': 'kitten', 'dog': 'puppy'})

Teraz nowe wartości zostaną zadane za pomocą funkcji. Na przykład tak:

In [9]:
s.map('I am a {}'.format, na_action='ignore')

0    I am a A
1    I am a B
2    I am a C
3    I am a D
dtype: object

albo tak, używając wyrażenia lambda:

In [8]:
pd.Series([1,2,3,4,5]).map(lambda x: x+2)

0    3
1    4
2    5
3    6
4    7
dtype: int64

No i jeszcze zadamy nowe wartości za pomocą innej serii:

In [7]:
s = pd.Series(['A','B','C','D'])
other_series = pd.Series(range(len(s)), index = ['C', 'D', 'B', 'A'])
s.map(other_series)

0    3
1    2
2    0
3    1
dtype: int64

### Ćwiczenie 1

Utworzyć serię danych ciśnienia atmosferycznego w hPa w pierwszych siedmiu dniach roku 2024 w Warszawie. Wartości liczbowe można pobrać [stąd](https://www.ekologia.pl/pogoda/polska/mazowieckie/warszawa/archiwum,zakres,01-01-2024_07-01-2024).

In [ ]:
dni_index = 
s = pd.Series([1004,2323.1,2312,2])

---

### Ramka danych w pandas

Teraz przejdziemy do omówienia klasy [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). Obiekty tej klasy to dwuwymiarowe tablice danych z indeksami kolumn i wierszy, gdzie poszczególne kolumny to serie danych, tj. obiekty klasy [pandas.Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html).

Na początek utworzymy obiekt DataFrame na podstawie słownika:

In [10]:
data = {'Kraj': ['Belgia', 'Indie', 'Brazylia'],
        'Stolica': ['Bruksela', 'New Delhi', 'Brasilia'],
        'Populacja': [11190846, 1303171035, 207847528]}
df = pd.DataFrame(data,columns=['Kraj', 'Stolica', 'Populacja'])
df

,Kraj,Stolica,Populacja
0,Belgia,Bruksela,11190846
1,Indie,New Delhi,1303171035
2,Brazylia,Brasilia,207847528


Sprawdźmy indkes kolumn tego obiektu:

In [11]:
df.columns

Index(['Kraj', 'Stolica', 'Populacja'], dtype='object')

Listę zawierającą nazwy kolumn możemy uzyskać w ten sposób:

In [12]:
df.columns.tolist()

['Kraj', 'Stolica', 'Populacja']

A teraz zobaczmy czym jest indeks wierszy:

In [ ]:
df.index

Sprawdźmy jeszcze obiektem jakiej klasy jest jedna z kolumn:

In [ ]:
type(df['Kraj'])

Kolumny są obiektami klasy [pandas.Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html). Ale elementy poszczególnych kolumn już mogą być różnych typów. Dlatego też dla każdej kolumny DataFrame przechowywany jest jej typ danych. Informacje o tym typie możemy uzyskać za pomocą atrybutu dtypes:

In [ ]:
print(df.dtypes)

Możemy też utworzyć obiekt klasy DataFrame łącząc serie danych:

Przykład 1:

In [13]:
s1 = pd.Series(range(6))
s2 = pd.Series(range(6,12))
df = pd.concat([s1, s2], axis=1)
df

,0,1
0,0,6
1,1,7
2,2,8
3,3,9
4,4,10
5,5,11


Przykład 2: Różne indeksy dla poszczególnych serii danych

In [14]:
s1 = pd.Series(range(6))
s2 = s1 ** s1
# Zmieniamy indeks s2
s2.index = s2.index + 3
df = pd.concat([s1, s2], axis=1)
df

,0,1
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,1.0
4,4.0,1.0
5,5.0,4.0
6,NaN,27.0
7,NaN,256.0
8,NaN,3125.0


Przykład 3:

In [ ]:
s3 = pd.Series({'Tomek':1, 'Ala':4, 'Ola':9})
s4 = pd.Series({'Kasia':3, 'Ala':2, 'Tomek':5})
df = pd.concat({'A':s3, 'B':s4}, axis=1)
df

Co mogliśmy zaobserwować?

- Indeksy poszczególnych serii danych były uzgadniane a brakujące wartości dla poszczególnych elementów indeksu uzupełniane NaN.
- Kolejność elementów w wynikowym obiekcie klasy DataFrame mogła ulec zmianie w stosunku do kolejności w wejściowych seriach danych.

Obiekt klasy DataFrame możemy też utworzyć wczytując dane z pliku, np. w formacie CSV:

In [ ]:
import csv

with open('kraje_dane.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["Kraj", "Stolica", "Populacja"]
    writer.writerow(field)
    writer.writerow(['Belgia', 'Bruksela', 11190846])
    writer.writerow(['Indie', 'New Delhi', 1303171035])
    writer.writerow(['Brazylia', 'Brasilia', 207847528])

In [ ]:
df = pd.read_csv('kraje_dane.csv', header=0)
df

Obiekt klasy DataFrame możemy skonwertować to tablicy NumPy za pomocą metody pandas.DataFrame.to_numpy():

In [ ]:
df_np = df.to_numpy()
df_np

lub za pomocą atrybutu [pandas.DataFrame.values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.values.html):

In [105]:
df.values

array([[390.],
       [350.],
       [ 30.],
       [ 20.]])

W łatwy sposób możemy podmienić zawartość kolumn w obiekcie DataFrame:

In [106]:
df[['Stolica', 'Populacja']] = df[['Populacja', 'Stolica']]
df

KeyError: "None of [Index(['Populacja', 'Stolica'], dtype='object')] are in the [columns]"

Tylko, że w tym przypadku nie ma to sensu. Dlatego przywracamy pierwotną postać:

In [107]:
df[['Stolica', 'Populacja']] = df[['Populacja', 'Stolica']]
df

KeyError: "None of [Index(['Populacja', 'Stolica'], dtype='object')] are in the [columns]"

---
Wczytamy teraz słynny zbiór danych z pomiarami kwiatów irysa, udostępniony po raz pierwszy przez Ronalda Fishera w roku 1936. Jest to jeden z najbardziej znanych zbiorów w analizie danych, stosowany do konstruowania modeli do rozwiązywania zadania klasyfikacji. Zbiór jest udostępniony m. in. w repozytorium [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/53/iris).

Zbiór składa się ze 150 obserwacji, po 50 dla każdego z trzech gatunków kwiatów irysa: irys (kosaciec) szczecinkowaty (ang. *setosa*), irys wirginijski (ang. *virginica*) i irys różnobarwny (ang. *versicolor*). Mierzone były
4 cechy (w centymetrach): długości i szerokości działki kielicha (ang. *sepal*) oraz płatka (ang. *petal*).

<center>
<div>
<img src="img/iris1.png" width="400"><img src="img/iris2.jpg" width="150">
</div>
</center>

In [108]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(url, header=None)
df.columns = ['sepal length','sepal width','petal length','petal width','class']

Wypiszmy informacje o utworzonej ramce danych. 

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  150 non-null    float64
 1   sepal width   150 non-null    float64
 2   petal length  150 non-null    float64
 3   petal width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


W szczególności widzimy, że nasz zbiór danych nie zawiera brakujących danych.

Wypiszmy pierwszych kilka wierszy:

In [110]:
df.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


i kilka ostatnich wierszy: 

In [111]:
df.tail()

,sepal length,sepal width,petal length,petal width,class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


W łatwy sposób możemy też policzyć podstawowe statystyki dla poszczególnych kolumn zawierających dane liczbowe:

In [112]:
df.describe()

,sepal length,sepal width,petal length,petal width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Mamy możliwość wybrania pojedynczej kolumny

- do serii danych:

In [113]:
df['sepal length']

0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: sepal length, Length: 150, dtype: float64

- do ramki danych (zwróćmy uwagę na użycie podwójnego nawiasowania):

In [114]:
df[['sepal length']]

,sepal length
0,5.1
1,4.9
2,4.7
3,4.6
4,5.0
...,...
145,6.7
146,6.3
147,6.5
148,6.2


W przypadku wyboru więcej niż jednej kolumny, musimy użyć podwójnego nawiasowania: 

In [115]:
df[['sepal length', 'petal width']]

,sepal length,petal width
0,5.1,0.2
1,4.9,0.2
2,4.7,0.2
3,4.6,0.2
4,5.0,0.2
...,...,...
145,6.7,2.3
146,6.3,1.9
147,6.5,2.0
148,6.2,2.3


To już nie zadziała:

In [116]:
df['sepal length', 'petal width']

KeyError: ('sepal length', 'petal width')

Możemy dokonywać wyboru kolumn za pomocą zakresu etykiet lub wartości boolowskich:

In [117]:
df.loc[:, 'sepal length':'petal length']

,sepal length,sepal width,petal length
0,5.1,3.5,1.4
1,4.9,3.0,1.4
2,4.7,3.2,1.3
3,4.6,3.1,1.5
4,5.0,3.6,1.4
...,...,...,...
145,6.7,3.0,5.2
146,6.3,2.5,5.0
147,6.5,3.0,5.2
148,6.2,3.4,5.4


In [118]:
df.loc[:, [False, False, True, False, True]]

,petal length,class
0,1.4,Iris-setosa
1,1.4,Iris-setosa
2,1.3,Iris-setosa
3,1.5,Iris-setosa
4,1.4,Iris-setosa
...,...,...
145,5.2,Iris-virginica
146,5.0,Iris-virginica
147,5.2,Iris-virginica
148,5.4,Iris-virginica


Wiersze mogą być wybierane za pomocą wartości indeksu:

In [119]:
idx = df[df['petal length'] <= 2].index
print(df.loc[idx])

    sepal length  sepal width  petal length  petal width        class
0            5.1          3.5           1.4          0.2  Iris-setosa
1            4.9          3.0           1.4          0.2  Iris-setosa
2            4.7          3.2           1.3          0.2  Iris-setosa
3            4.6          3.1           1.5          0.2  Iris-setosa
4            5.0          3.6           1.4          0.2  Iris-setosa
5            5.4          3.9           1.7          0.4  Iris-setosa
6            4.6          3.4           1.4          0.3  Iris-setosa
7            5.0          3.4           1.5          0.2  Iris-setosa
8            4.4          2.9           1.4          0.2  Iris-setosa
9            4.9          3.1           1.5          0.1  Iris-setosa
10           5.4          3.7           1.5          0.2  Iris-setosa
11           4.8          3.4           1.6          0.2  Iris-setosa
12           4.8          3.0           1.4          0.1  Iris-setosa
13           4.3    

In [120]:
df['petal length'] <= 2

0       True
1       True
2       True
3       True
4       True
       ...  
145    False
146    False
147    False
148    False
149    False
Name: petal length, Length: 150, dtype: bool

Możemy dostawać się do konkretnych elementów ramki danych za pomocą standardowych indeksów całkowitoliczbowych korzystając z metody [pandas.DataFrame.iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html):

In [121]:
sub_df = df.iloc[:5,:5]
sub_df

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Możemy też zmienić wartość konkretnego elementu: 

In [122]:
sub_df.iloc[2,1] = -3.2
sub_df

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,-3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Zobaczmy czy nasze przypisanie spowodowało zmiany w df:

In [123]:
df.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Nie, a więc df.iloc utworzyło kopię zadanego fragmentu ramki danych!

Możemy usunąć kolumnę z ramki danych w następujący sposób:

In [124]:
removed_col = sub_df.pop('sepal width')
removed_col

0    3.5
1    3.0
2   -3.2
3    3.1
4    3.6
Name: sepal width, dtype: float64

In [125]:
sub_df

,sepal length,petal length,petal width,class
0,5.1,1.4,0.2,Iris-setosa
1,4.9,1.4,0.2,Iris-setosa
2,4.7,1.3,0.2,Iris-setosa
3,4.6,1.5,0.2,Iris-setosa
4,5.0,1.4,0.2,Iris-setosa


Możemy iterować po kolumnach ramki danych:

In [126]:
for (name, series) in df.items():
    print('Nazwa kolumny: ' + str(name))
    print('Trzecia wartość w kolumnie: ' + str(series.iat[2]) + '\n')

Nazwa kolumny: sepal length
Trzecia wartość w kolumnie: 4.7

Nazwa kolumny: sepal width
Trzecia wartość w kolumnie: 3.2

Nazwa kolumny: petal length
Trzecia wartość w kolumnie: 1.3

Nazwa kolumny: petal width
Trzecia wartość w kolumnie: 0.2

Nazwa kolumny: class
Trzecia wartość w kolumnie: Iris-setosa



lub po wierszach:

In [127]:
for (row_index, series) in df.iloc[:5,:].iterrows():
    print(f"Indeks wiersza: {row_index}")
    print(f"Pierwszy element w wierszu: {series.iat[0]}")

Indeks wiersza: 0
Pierwszy element w wierszu: 5.1
Indeks wiersza: 1
Pierwszy element w wierszu: 4.9
Indeks wiersza: 2
Pierwszy element w wierszu: 4.7
Indeks wiersza: 3
Pierwszy element w wierszu: 4.6
Indeks wiersza: 4
Pierwszy element w wierszu: 5.0


---

## 2. Przykłady operacji matematycznych na kolumnach ramki danych w pandas

Suma elementów w kolumnie

In [128]:
df['petal length'].sum()

np.float64(563.8)

Iloczyn elementów w kolumnie

In [129]:
df['petal length'].prod()

np.float64(3.774489440906495e+76)

Najmniejsza wartość w kolumnie

In [130]:
df['petal length'].min()

np.float64(1.0)

Wartość średnia wartości w kolumnie

In [131]:
df['petal length'].mean()

np.float64(3.758666666666666)

Mediana wartości w kolumnie

In [132]:
df['petal length'].median()

np.float64(4.35)

Podstawowe statystyki dla wartości w kolumnie

In [133]:
df['petal length'].describe()

count    150.000000
mean       3.758667
std        1.764420
min        1.000000
25%        1.600000
50%        4.350000
75%        5.100000
max        6.900000
Name: petal length, dtype: float64

Indeks wierszowy dla wartości minimalnej w kolumnie

In [134]:
df['petal length'].idxmin()

22

i dla wartości maksymalnej

In [135]:
df['petal length'].idxmax()

118

---
## 3. Przykłady sortowania ramki danych w pandas

In [136]:
df = pd.DataFrame({'col1': ['A', 'A', 'B', np.nan, 'D', 'C'],
                   'col2': [2, 1, 9, 8, 7, 4],
                   'col3': [0, 1, 9, 4, 2, 3],
                   #'col4': ['a', 'B', 'c', 'D', 'e', 'F']
                   'col4': ['b', 'a', 'c', 'D', 'e', 'F']
})
df

,col1,col2,col3,col4
0,A,2,0,b
1,A,1,1,a
2,B,9,9,c
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F


Sortowanie ramki danych według wybranej kolumny

In [137]:
df.sort_values(by=['col4'])

,col1,col2,col3,col4
3,NaN,8,4,D
5,C,4,3,F
1,A,1,1,a
0,A,2,0,b
2,B,9,9,c
4,D,7,2,e


Sortowanie ramki danych według kilku kolumn

In [138]:
df.sort_values(by=['col1', 'col2'])

,col1,col2,col3,col4
1,A,1,1,a
0,A,2,0,b
2,B,9,9,c
5,C,4,3,F
4,D,7,2,e
3,NaN,8,4,D


Sortowanie malejąco według określonej kolumny z wartościami NaN na pierwszych pozycjach:

In [139]:
df.sort_values(by='col1', ascending=False, na_position='first')

,col1,col2,col3,col4
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F
2,B,9,9,c
0,A,2,0,b
1,A,1,1,a


Sortowanie z wykorzystaniem funkcji zwracającej wartość klucza:

In [164]:
df.sort_values(by='col4', key=lambda col: col.str.lower())

,col1,col2,col3,col4
1,A,1,1,a
0,A,2,0,b
2,B,9,9,c
3,NaN,8,4,D
4,D,7,2,e
5,C,4,3,F


In [141]:
df['col4'].str.lower()

0    b
1    a
2    c
3    d
4    e
5    f
Name: col4, dtype: object

In [142]:
df.sort_values(by=['col1','col4'], key=lambda col: col.str.lower())

,col1,col2,col3,col4
1,A,1,1,a
0,A,2,0,b
2,B,9,9,c
5,C,4,3,F
4,D,7,2,e
3,NaN,8,4,D


**Uwaga:** Argumentami funkcji lambda są całe serie danych odpowiadające poszczególnym kolumnom z listy 'by' a nie poszczególne elementy w kolumnach. Możemy to zaobserwować tutaj:

In [165]:
df.sort_values(by=['col1','col4'], key=lambda col: (print(col),col.str.lower())[1])

0      A
1      A
2      B
3    NaN
4      D
5      C
Name: col1, dtype: object
0    b
1    a
2    c
3    D
4    e
5    F
Name: col4, dtype: object


,col1,col2,col3,col4
1,A,1,1,a
0,A,2,0,b
2,B,9,9,c
5,C,4,3,F
4,D,7,2,e
3,NaN,8,4,D


Argument key jest podobny do tego we wbudowanej funkcji [sorted()](https://docs.python.org/3/library/functions.html#sorted). Ale jest pewna istotna różnica. Funkcja podawana jako key w przypadku df.sort_values() powinna być zwektoryzowana. Powinna przyjmować serię danych jako argument i zwracać serię danych o tym samym kształcie co argument. Funkcja ta jest aplikowana niezależnie do każdej kolumny wymienionej w by.

### Ćwiczenie 2

Dana jest ramka danych oraz lista zawierająca wszystkie unikatowe wartości jednej z kolumn ramki. Posortuj wiersze ramki tak, aby wartości w tej kolumnie występowały w tym samym porządku co na podanej liście.

Przykład:
Dla ramki danych

<table>
  <tr>
    <th>colA</th>
    <th>colB</th>
  </tr>
  <tr>
    <th>A</th>
    <th>1</th>
  </tr>
  <tr>
    <th>B</th>
    <th>2</th>
  </tr>
  <tr>
    <th>C</th>
    <th>3</th>
  </tr>
  <tr>
    <th>D</th>
    <th>4</th>
  </tr>
</table>

i listy ['C','A','D','B'] dla kolumny colA wynikiem powinno być

<table>
  <tr>
    <th>colA</th>
    <th>colB</th>
  </tr>
  <tr>
    <th>C</th>
    <th>3</th>
  </tr>
  <tr>
    <th>A</th>
    <th>1</th>
  </tr>
  <tr>
    <th>D</th>
    <th>4</th>
  </tr>
  <tr>
    <th>B</th>
    <th>2</th>
  </tr>
</table>

---
## 4. Złączenia ramek danych w pandas

Istnieją następujące sposoby złączania ramek danych w pandas:
- za pomocą metod [pandas.DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) lub [pandas.DataFrame.join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join) (bazodanowe SQL join);
- za pomocą funkcji [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) (dołączanie kolumn lub wierszy);
- za pomocą metody [pandas.DataFrame.combine_first](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html).

In [144]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    
                    'value': [1, 2, 3, 5]})

df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],

                    'value': [5, 6, 7, 8]})

Bazodanowe złączenie po indeksach (SQL full outer join):

In [145]:
df_merged = pd.merge(left=df1, right=df2, left_index=True, right_index=True)
df_merged

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,bar,2,bar,6
2,baz,3,baz,7
3,foo,5,foo,8


Bazodanowe złączenie po określonych kolumnach (kluczach):

In [146]:
df1.merge(df2, how='left', left_on='lkey', right_on='rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,1,foo,8
2,bar,2,bar,6
3,baz,3,baz,7
4,foo,5,foo,5
5,foo,5,foo,8


In [147]:
df1.merge(df2, how='right', left_on='lkey', right_on='rkey')

,lkey,value_x,rkey,value_y
0,foo,1,foo,5
1,foo,5,foo,5
2,bar,2,bar,6
3,baz,3,baz,7
4,foo,1,foo,8
5,foo,5,foo,8


Zwróćmy uwagę, jak zostały zmienione etykiety 'value' w odpowiednich kolumnach.

Przemianowywanie kolumn o takich samych etykietach można dostosować do własnych potrzeb:

In [148]:
df1.merge(df2, how='left', left_on='lkey', right_on='rkey', suffixes=('_left', '_right'))

,lkey,value_left,rkey,value_right
0,foo,1,foo,5
1,foo,1,foo,8
2,bar,2,bar,6
3,baz,3,baz,7
4,foo,5,foo,5
5,foo,5,foo,8


Przyjrzymy się teraz różnym rodzajom złączeń bazodanowych:

In [149]:
df1 = pd.DataFrame({'a': ['foo', 'bar'], 'b': [1, 2]})
df2 = pd.DataFrame({'a': ['foo', 'baz'], 'c': [3, 4]})
print(df1)
print(df2)

     a  b
0  foo  1
1  bar  2
     a  c
0  foo  3
1  baz  4


Odpowiednik 'SQL full outer join'; używa teoriomnogościowej sumy zbiorów kluczy, która jest sortowana leksykograficznie:

In [150]:
df1.merge(df2, how='outer', on='a')

,a,b,c
0,bar,2.0,NaN
1,baz,NaN,4.0
2,foo,1.0,3.0


Odpowiednik 'SQL inner join'; używa teoriomnogościwego iloczynu (przecięcia) zbiorów kluczy, zachowując kolejność kluczy w lewej ramce:

In [151]:
#pd.merge(left=df1, right=df2, how='inner', on='a')
df1.merge(df2, how='inner', on='a')

,a,b,c
0,foo,1,3


Odpowiednik 'SQL left outer join'; używa wyłącznie kluczy z lewej ramki zachowując ich kolejność:

In [152]:
#pd.merge(left=df1, right=df2, how='left', on='a')
df1.merge(df2, how='left', on='a')

,a,b,c
0,foo,1,3.0
1,bar,2,NaN


Odpowiednik 'SQL right outer join', używa wyłącznie kluczy prawej ramki zachowując ich kolejność:

In [153]:
#pd.merge(left=df1, right=df2, how='right', on='a')
df1.merge(df2, how='right', on='a')

,a,b,c
0,foo,1.0,3
1,baz,NaN,4


Iloczyn kartezjański obu ramek danych, który zachowuje kolejność kluczy lewej ramki:

In [166]:
df1.merge(df2, how='cross')

,A,B_x,B_y,C
0,NaN,4.0,3,1
1,NaN,4.0,3,1
2,0.0,NaN,3,1
3,0.0,NaN,3,1


Złączenia można realizować także za pomocą metody [pandas.DataFrame.join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join), która domyślnie realizuje złączenie po indeksach (w odróżnieniu od merge, która domyślnie dokonuje złączenia po wspólnych kolumnach):

In [167]:
df1.join(other=df2, how='outer', lsuffix='_lewa', rsuffix='_prawa')

,A,B_lewa,B_prawa,C
0,NaN,4.0,NaN,NaN
1,0.0,NaN,3.0,1.0
2,NaN,NaN,3.0,1.0


Następująca próba złączenia nie zadziała, gdyż próbuje dokonać złączenia na kolumnie df1.'a' i indeksie df2, których elementy są różnych typów: 

In [168]:
df1.join(other=df2, on=['a'], how='outer', lsuffix='_lewa', rsuffix='_prawa')

KeyError: 'a'

Za pomocą funkcji [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) możemy połączyć ramki danych w jedną ramkę kolumnami:

In [169]:
pd.concat([df1,df2],axis=1)

,A,B,B,C
0,NaN,4.0,NaN,NaN
1,0.0,NaN,3.0,1.0
2,NaN,NaN,3.0,1.0


bądź wierszami:

In [170]:
pd.concat([df1,df2],axis=0)

,A,B,C
0,NaN,4.0,NaN
1,0.0,NaN,NaN
1,NaN,3.0,1.0
2,NaN,3.0,1.0


Przy wierszowym łączeniu ramek przydatna może okazać się możliwość ignorowania wartości indeksów łączonych ramek danych. W takim przypadku kolejne wiersze będą inkesowane liczbami od 0 do n-1:

In [159]:
pd.concat([df1,df2],axis=0,ignore_index=True)

,a,b,c
0,foo,1.0,NaN
1,bar,2.0,NaN
2,foo,NaN,3.0
3,baz,NaN,4.0


Domyślną wartością parametru join jest 'outer'. Możemy wywołać funkcję concat z argumentem join o wartości 'inner', co spowoduje uwzględnienie w wyniku naszego wywołania tylko kolumn o wspólnych etykiach w łączonych ramkach danych:

In [160]:
pd.concat([df1,df2],axis=0,join='inner')

,a
0,foo
1,bar
0,foo
1,baz


W przypadku łaczenia wierszy zadziała to tak:

In [161]:
df1 = pd.DataFrame({'a': ['foo', 'bar', 'abc'], 'b': [1, 2, 7]})
df2 = pd.DataFrame({'a': ['foo', 'baz'], 'c': [3, 4]})
print(df1)
print(df2)
pd.concat([df1,df2],axis=1,join='inner')

     a  b
0  foo  1
1  bar  2
2  abc  7
     a  c
0  foo  3
1  baz  4


,a,b,a,c
0,foo,1,foo,3
1,bar,2,baz,4


Metoda [pandas.DataFrame.combine_first](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html) pozwala na uzupełnianie brakujących wartości (NaN) danej ramki danych odpowiednimi wartościami z drugiej ramki danych:

In [162]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
print(df1)
print(df2)
df1.combine_first(df2)

     A    B
0  NaN  NaN
1  0.0  4.0
   A  B
0  1  3
1  1  3


,A,B
0,1.0,3.0
1,0.0,4.0


Wywołanie df1.combine_first(df2) łączy dwie ramki danych uzupełniając brakujące wartości w lokacjach ramki df1 istniejącymi wartościami w odpowiadających lokacjach ramki df2. Brakująca wartość pozostanie w df1 jeżeli nie ma określonej wartości w odpowiadającej lokacji w ramce df2.

Indeksy kolumn i wierszy wynikowej ramki danych są teoriomnogościową sumą odpowiednich indeksów obu ramek.

In [163]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
print(df1)
print(df2)
df1.combine_first(df2)

     A    B
0  NaN  4.0
1  0.0  NaN
   B  C
1  3  1
2  3  1


,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


---
## 5. Grupowanie w pandas

Mechanizm grupowania w pandas pozwala podzielić dane na grupy i zastosować funkcję agregującą do każdej z grup niezależnie.

Metoda [pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) zwraca obiekt groupby zawierający informacje o utworzonych grupach:

In [73]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon', 'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
df

,Animal,Max Speed
0,Falcon,380.0
1,Falcon,370.0
2,Parrot,24.0
3,Parrot,26.0


Grupowanie za pomocą kolumny grupującej:

In [74]:
gb = df.groupby(['Animal'])
gb

Wartością atrubytu .groups jest słownik reprezentujący grupy, gdzie klucze to wartości z kiolumny grupującej a wartości to odpowiadające tym wartościom wiersze:

In [75]:
gb.groups

{'Falcon': [0, 1], 'Parrot': [2, 3]}

Iteracja po grupach (zwykle nie jest używana):

In [76]:
for name, group in gb:
    print (name)
    print (group)

('Falcon',)
   Animal  Max Speed
0  Falcon      380.0
1  Falcon      370.0
('Parrot',)
   Animal  Max Speed
2  Parrot       24.0
3  Parrot       26.0


In [77]:
# gb.get_group('Parrot') # Starsze wersje pandas
gb.get_group(('Parrot',)) # W przyszłych wersjach pandas

,Animal,Max Speed
2,Parrot,24.0
3,Parrot,26.0


Możemy teraz policzyć średnie prędkości uzyskiwane przez poszczególne zwierzęta stosując funkcję agregującą, w naszym przypadku funkcję mean():

In [78]:
gb.mean()

,Max Speed
Animal,
Falcon,375.0
Parrot,25.0


Funkcja mean() zostanie zastosowana do każdej kolumny w każdej z grup niezależnie. Zobaczymy to jeszcze poniżej.

Możemy też zastosować wiele funkcji agregujących do jednej kolumny.

In [79]:
#df_result = gb.agg([np.sum, np.mean, np.std])
df_result = gb.agg(["sum", "mean", "std"]) # W przyszłych wersjach pandas
df_result

Max Speed                 
             sum   mean       std
Animal                           
Falcon     750.0  375.0  7.071068
Parrot      50.0   25.0  1.414214

Wynikowa ramka danych posiada indeks hierarchiczny ([MultiIndex](https://pandas.pydata.org/docs/user_guide/advanced.html)) dla kolumn:

In [80]:
df_result.columns

MultiIndex([('Max Speed',  'sum'),
            ('Max Speed', 'mean'),
            ('Max Speed',  'std')],
           )

Możemy też zastosować wiele (różnych) funkcji agregujących do wielu kolumn: 

In [81]:
l = [[1, 2, 3], [1, 5, 4], [2, 0, 3], [1, 2, 2]]
df_abc = pd.DataFrame(l, columns=["a", "b", "c"])
df_abc

,a,b,c
0,1,2,3
1,1,5,4
2,2,0,3
3,1,2,2


In [82]:
#df_abc.groupby('a').agg({'b': np.count_nonzero, 'c': [np.mean, np.sum]})
df_abc.groupby('a').agg({'b': np.count_nonzero, 'c': ["mean", "sum"]}) # W przyszłości

b    c    
  count_nonzero mean sum
a                       
1             3  3.0   9
2             0  3.0   3

Jeżeli kluczem grupy jest NA, wiersze/kolumny odpowiadające takiej grupie domyślnie zostaną pominięte. Aby to zmienić, należy wartość argumentu dropna ustawić na False. Przeanalizujemy to na przykładzie:

In [83]:
l = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df = pd.DataFrame(l, columns=["a", "b", "c"])
df

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


Pogrupujemy po kolumnie 'b' z domyślną wartością argumentu dropna:

In [84]:
df.groupby(by=["b"]).sum()

,a,c
b,,
1.0,2,3
2.0,2,5


W powyższym przykładzie funkcja agregującą sum() została zastosowana do wszystkich kolumn, tj. a i c, dla każdej z grup niezależnie. Możemy to zmienić wybierając kolumny, do których chcemy zastosować funkcję agregującą:

In [85]:
df.groupby(by=["b"])[['c']].sum()

,c
b,
1.0,3
2.0,5


Teraz zmienimy wartość dropna na False:

In [86]:
df.groupby(by=["b"], dropna=False).sum()

,a,c
b,,
1.0,2,3
2.0,2,5
NaN,1,4


Możemy również grupować za pomocą poszczególnych poziomów indeksu hierarchicznego.

Najpierw utworzymy indeks hierarchiczny z tablicy:

In [87]:
arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],
          ['Captive', 'Wild', 'Captive', 'Wild']]
index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))
index

MultiIndex([('Falcon', 'Captive'),
            ('Falcon',    'Wild'),
            ('Parrot', 'Captive'),
            ('Parrot',    'Wild')],
           names=['Animal', 'Type'])

Następnie utworzymy ramkę danych z naszymi danymi indeksowaną indeksem hierarchicznym:

In [88]:
df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.]}, index=index)
df

Max Speed
Animal Type              
Falcon Captive      390.0
       Wild         350.0
Parrot Captive       30.0
       Wild          20.0

Pogrupujemy dane na podstawie pierwszego poziomu indeksu hierarchicznego:

In [89]:
df.groupby(level=0).groups

{'Falcon': [('Falcon', 'Captive'), ('Falcon', 'Wild')], 'Parrot': [('Parrot', 'Captive'), ('Parrot', 'Wild')]}

a teraz na podstawie poziomu drugiego:

In [90]:
df.groupby(level='Type').groups

{'Captive': [('Falcon', 'Captive'), ('Parrot', 'Captive')], 'Wild': [('Falcon', 'Wild'), ('Parrot', 'Wild')]}

---
## 6. Czas w pandas

Tak jak było zapowiedziane wcześniej, na koniec wrócimy jeszcze do zagadnień związanych z czasem w pandas.

Pandas wprowadza trzy główne pojęcia związane z czasem:
- Datetimes - konkretne daty i czasy wraz z informacją o strefie czasowej;
- Timedeltas - bezwględne czasy trwania;
- Timespans - przedziały czasowe określone poprzez czasy początkowy i końcowy oraz krok czasowy.
- 
Podstawową klasą jest klasa [pandas.Timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp), będąca podklasą klasy [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime.datetime) z biblioteki standardowej. Pozwala na tworzenie obiektów reprezentujących konkretną datę i czas wraz z informacją o strefie czasowej.

In [91]:
pd.Timestamp(year=2023, month=12, day=17, hour=11)

Timestamp('2023-12-17 11:00:00')

Czas może być bardzo dokładny, tzn. określony z dokładnością co do nanosekundy:

In [92]:
pd.Timestamp('2020-03-14T15:32:52.192548651', tz='UTC')

Timestamp('2020-03-14 15:32:52.192548651+0000', tz='UTC')

Zobaczmy jak można konwertować czas z uwzględnieniem informacji o strefie czasowej:

In [93]:
ts = pd.Timestamp(year=2023, month=10, day=17, hour=11, tz="Europe/Warsaw")
ts

Timestamp('2023-10-17 11:00:00+0200', tz='Europe/Warsaw')

Konwersji można dokonać za pomocą metody [Timestamp.tz_convert()](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.tz_convert.html#pandas.Timestamp.tz_convert):

In [94]:
ts.tz_convert("US/Eastern")

Timestamp('2023-10-17 05:00:00-0400', tz='US/Eastern')

albo [Timestamp.astimezone()](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.astimezone.html#pandas.Timestamp.astimezone):

In [95]:
ts.astimezone("US/Eastern")

Timestamp('2023-10-17 05:00:00-0400', tz='US/Eastern')

Możemy też wypisać datę i czas obiektu Timestamp w formacie POSIX, tj. w postaci czasu uniksowego, czyli liczbę sekund od początku 1970 roku [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), tj. od chwili zwanej początkiem epoki Uniksa (ang. *Unix Epoch*), ale bez uwzględnienia [sekund przestępnych](https://pl.wikipedia.org/wiki/Sekunda_przest%C4%99pna):

In [96]:
ts.timestamp()

1697533200.0

Ciekawostka: możemy nawet wypisać datę i czas w postaci [dni juliańskich](https://pl.wikipedia.org/wiki/Data_julia%C5%84ska), czyli liczby dni, która upłynęła od godziny 12:00 czasu uniwersalnego (czasu południka zerowego) w dniu 1 stycznia roku 4713 p.n.e. według kalendarza juliańskiego (przedłużonego odpowiednio wstecz):

In [97]:
ts.to_julian_date()

np.float64(2460234.9583333335)

Możemy odejmować obiekty klasy Timestamp jeżeli strefy czasowe obu obiektów są określone:

In [98]:
teraz = ts.today()
print(teraz)

2024-11-25 15:02:25.383988


To nie zadziała:

In [99]:
teraz - ts

TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects.

ale po dookreśleniu strefy czasowej dla teraz już tak:

In [100]:
teraz = teraz.tz_localize(tz="Europe/Warsaw")

dt = teraz - ts
dt

Timedelta('405 days 05:02:25.383988')

W rezultacie dostajemy obiekt klasy [Timedelta](https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html). I możemy go wykorzystać np. w taki oto sposób:

In [101]:
pd.Timestamp('2020-03-14T15:32:52.192548651', tz='UTC') + dt

Timestamp('2021-04-23 20:35:17.576536651+0000', tz='UTC')